In [1]:
a = np.array([[1,2],[3,4],[5,6]])

In [2]:
print(a.shape)

(3, 2)


In [3]:
a

array([[1, 2],
       [3, 4],
       [5, 6]])

In [7]:
a.reshape(2,-1)   #나머지는 몇개인지 모르겠다 -> -1 쓰면 됨

array([[1, 2, 3],
       [4, 5, 6]])

In [8]:
b = np.array([
    [1,2,3,4],
    [3,4,5,6],
    [5,6,7,8]])

In [9]:
b.shape

(3, 4)

In [10]:
b = np.array([
    [[1,2],[3,4]],
    [[3,4],[5,6]],
    [[5,6],[7,8]]])

In [11]:
b.shape

(3, 2, 2)

In [12]:
b.reshape(2,-1)

array([[1, 2, 3, 4, 3, 4],
       [5, 6, 5, 6, 7, 8]])

In [13]:
a,b=(7,3)      #위 표현이 더 좋아보여
print(a)
print(b)

c=(7,3)
print(c[0])
print(c[1])

7
3
7
3


In [14]:
np.max([1,2,3,4,1,8,2.4])

8.0

In [15]:
np.argmax([1,2,3,4,1,8,2.4])  #최댓값에 해당하는 index

5

In [16]:
np.argmax([0,0,0,0,1,0])    #one-hot encoding

4

In [18]:
np.argmax([0,0,0,1,1,0])

3

In [20]:
label = [0,0,2,1]
x = np.array([[1,0,0],[1,0,0],[0,0,1],[0,1,0]])
#to_categorical 함수: label to one-hot encoding
#반대는 argmax 가 그 역할 해줌

In [21]:
np.argmax(x)

0

In [22]:
np.argmax(x,axis=1)  #axis: 0:, 1:row sum, col sum

array([0, 0, 2, 1], dtype=int64)

In [3]:
a = np.array([[1,2],[3,4],[5,6]])
print(a)
b = a.reshape(3,2,1)  #reshape 시 중요한 건 당연히 개수가 같아야지
print(b.shape)

[[1 2]
 [3 4]
 [5 6]]
(3, 2, 1)


In [4]:
b[:,:,0]

array([[1, 2],
       [3, 4],
       [5, 6]])

In [5]:
b[:,:,:]

array([[[1],
        [2]],

       [[3],
        [4]],

       [[5],
        [6]]])

ensemble test

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from keras.layers import Dropout
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))
# We use all training data and validate on all test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


In [4]:
def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, )))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    sgd = optimizers.SGD(lr = 0.001)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [7]:
model1 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 1)
model2 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 1)
model3 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 1)
#서로 다른 모델 3개. 특별히 미니배치 사이즈 지정 안 했으므로 전체 배치로.

In [8]:
ensemble_clf = VotingClassifier(estimators = [
    ('model1', model1), 
    ('model2', model2), 
    ('model3', model3)], voting = 'soft')
#voting: 다수결 방식. hard/soft.
#soft: validation accuracy 반영해서 가중치 부여해서 결정

In [9]:
ensemble_clf.fit(X_train, y_train)

Epoch 1/10
60000/60000 [==============================] - 2s 41us/step - loss: 2.3383 - accuracy: 0.1100
Epoch 2/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.3014 - accuracy: 0.1124
Epoch 3/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.3003 - accuracy: 0.1124
Epoch 4/10
60000/60000 [==============================] - 2s 37us/step - loss: 2.2994 - accuracy: 0.1124
Epoch 5/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.2986 - accuracy: 0.1124
Epoch 6/10
60000/60000 [==============================] - 2s 36us/step - loss: 2.2978 - accuracy: 0.1124
Epoch 7/10
60000/60000 [==============================] - 2s 35us/step - loss: 2.2969 - accuracy: 0.1124
Epoch 8/10
60000/60000 [==============================] - 2s 36us/step - loss: 2.2960 - accuracy: 0.1124
Epoch 9/10
60000/60000 [==============================] - 2s 36us/step - loss: 2.2952 - accuracy: 0.1124
Epoch 10/10
60000/60000 [==============================

VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002E63F354D48>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002E63F354D08>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002E63F354D88>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [14]:
# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))
# We use all training data and validate on all test data
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

IndexError: tuple index out of range

In [9]:
print(X_train.shape)
X_train

(60000, 28, 28)


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [10]:
print(y_train.shape)
y_train

(60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
print(X_train.shape)
X_train

(60000, 784)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
print(y_train.shape)
y_train

(60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)